In [33]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import warnings
warnings.filterwarnings("ignore", category=UserWarning)  # Ignore UserWarnings for cleaner output
from torchvision import datasets, transforms, models

# Define transforms for your dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load local custom dataset (assumes folder structure: root/class_x/xxx.png)
train_data = datasets.ImageFolder(root='data/train', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
train_data.classes

['0__Anger',
 '1__Contempt',
 '2__Disgust',
 '3__Fear',
 '4__Happy',
 '5__Neutral',
 '6__Sad',
 '7__Surprise']

In [ ]:
class FaceExpression(nn.Module):
    def __init__(self, num_classes=8, dropout_rate=0.3):
        super(FaceExpression, self).__init__()
        
# Initialize the model
model = FaceExpression(num_classes=len(train_data.classes))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()



In [ ]:
epochs=100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)    
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")


#clear gpu cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Save the trained model
with open("classes.txt", "w") as f:
    for cls in train_data.classes:
        f.write(f"{cls}\n")
torch.save(model.state_dict(), 'face_expression_model.pth')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x62720 and 1280x8)